In [ ]:
!nvidia-smi

In [ ]:
from os import path
import torch
from fast_baseline import ActMAD, NORM, DUA, MeanTeacher # 경로 지정하는것에 대한 fix 필요
from ttadapters.models.rcnn import FasterRCNNForObjectDetection, SwinRCNNForObjectDetection
from ttadapters.datasets import SHIFTDataset

In [ ]:
# Set CUDA Device Number
DEVICE_NUM = 6

from os import environ
environ["CUDA_VISIBLE_DEVICES"] = str(DEVICE_NUM)
environ["CUDA_VISIBLE_DEVICES"]

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"INFO: Using device - {device}")

In [ ]:
import os
os.chdir("/workspace/ptta") # os.chdir("/home/ubuntu/test-time-adapters")

In [ ]:
DATA_ROOT = path.join(".", "data")

## Model Load

In [ ]:
USE_SWIN_T_BACKBONE = False

In [ ]:
if USE_SWIN_T_BACKBONE:
    model = SwinRCNNForObjectDetection(dataset=SHIFTDataset)
else:
    model = FasterRCNNForObjectDetection(dataset=SHIFTDataset)

model.load_from(model.Weights.NATUREYOO, weight_key="model")
model.to(device)

## Baseline

In [ ]:
# ActMAD
actmad = ActMAD.load(
    model=model,
    data_root=DATA_ROOT,
    device=device,
    batch_size=4,
    learning_rate=0.001,
    clean_bn_extract_batch=8
)

norm = NORM.load(
    model=model,
    data_root=DATA_ROOT,
    device=device,
    batch_size=4,
    source_sum=128  # NORM 특유의 하이퍼파라미터
)

dua = DUA.load(
    model=model,
    data_root=DATA_ROOT,
    device=device,
    batch_size=4,
    decay_factor=0.94,
    mom_pre=0.01,
    min_momentum_constant=0.0001
)

mean_teacher = MeanTeacher.load(
    model=model,
    data_root=DATA_ROOT,
    device=device,
    batch_size=4,
    learning_rate=0.001,
    ema_alpha=0.99,          # Teacher 모델 EMA 계수
    conf_threshold=0.5,      # Pseudo-label 신뢰도 임계값
    fixmatch_n=2,            # FixMatch augmentation 횟수
    fixmatch_m=5             # FixMatch augmentation 강도
)

In [ ]:
# ActMAD
actmad_results = actmad.evaluate_all_tasks()

# NORM
norm_results = norm.evaluate_all_tasks()

# DUA
dua_results = dua.evaluate_all_tasks()

# Mean-Teacher
mean_teacher_results = mean_teacher.evaluate_all_tasks()

In [ ]:
results = actmad_results # norm_results, dua_results, mean_teacher_results

print("=== mAP@0.50:0.95 Summary ===")
for task, metrics in results.items():
    print(f"{task:10s}: {metrics['mAP@0.50:0.95']:.3f}")